## TODO

https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-fit

https://github.com/abhishekkrthakur/commonlit-pairwise-model



## Introduction

Transformers разные слои захватывают разные уровни представлений. Они изучают богатую иерархию лингвистической информации, то есть с поверхностными функциями на нижних уровнях, синтаксическими функциями на средних уровнях и семантическими функциями на более высоких уровнях.

<img src = 'http://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png'/>


Авторы BERT протестировали стратегии встраивания слов, подавая различные комбинации векторов в качестве входных характеристик в BiLSTM, используемый в задаче распознавания именованных сущностей, и наблюдая за полученными баллами F1. Объединение последних четырех слоев дало наилучшие результаты.

Частично это демонстрируется тем, что разные уровни BERT кодируют очень разные виды информации, поэтому соответствующая стратегия объединения будет меняться в зависимости от приложения, поскольку разные уровни кодируют разные виды информации. Это справедливо и для других вариантов.

In [28]:
from IPython.display import clear_output, Image
!pip install transformers
clear_output()

In [19]:
import re
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch import nn, optim
import transformers

path_tr = '/content/drive/MyDrive/CommonLit/input/train.csv'
path_test = '/content/drive/MyDrive/CommonLit/input/test.csv'
path_sub = '/content/drive/MyDrive/CommonLit/input/sample_submission.csv'

SEED =13
np.random.seed(SEED)
torch.manual_seed(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = transformers.AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path='roberta-base')
model_config = transformers.AutoConfig.from_pretrained(
    pretrained_model_name_or_path='roberta-base',
    num_labels=1)
model = transformers.AutoModel.from_pretrained(
    pretrained_model_name_or_path='roberta-base',
    config=model_config)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
df = pd.read_csv(path_tr)

In [17]:
txt = df.excerpt[:6].values
tkzr = tokenizer.batch_encode_plus(
    list(txt), # batch_text_or_text_pairs has to be a list (got <class 'numpy.ndarray'>)
    truncation=True,
    max_length=256,
    padding='max_length',
    add_special_tokens=True,
    return_attention_mask=True,
    return_token_type_ids=False,
    return_tensors='pt'
)          

In [79]:
out = model(**tkzr)
out

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.0676,  0.0801,  0.0089,  ..., -0.0234, -0.0443, -0.0295],
                                                        [-0.0136,  0.2424, -0.1040,  ..., -0.4047,  0.0952, -0.0992],
                                                        [ 0.0357, -0.0299,  0.1018,  ..., -0.2944,  0.0626, -0.1154],
                                                        ...,
                                                        [ 0.0610, -0.0185,  0.1139,  ...,  0.2247,  0.1071,  0.0041],
                                                        [ 0.0610, -0.0185,  0.1139,  ...,  0.2247,  0.1071,  0.0041],
                                                        [ 0.0610, -0.0185,  0.1139,  ...,  0.2247,  0.1071,  0.0041]],
                                               
                                                       [[-0.0829,  0.1356,  0.0139,  ..., -0.0217, -0.0603, -0.049

### pooler_output

In [27]:
out['pooler_output'].shape, out['last_hidden_state'].shape, nn.Linear(768, 1)(out['pooler_output']).shape

(torch.Size([6, 768]), torch.Size([6, 256, 768]), torch.Size([6, 1]))

In [33]:
reg_head = nn.Linear(768, 1)(out['pooler_output'])
reg_head

tensor([[0.0772],
        [0.0678],
        [0.0837],
        [0.0888],
        [0.0915],
        [0.0879]], grad_fn=<AddmmBackward>)

### last_hidden_state
<img src = 'https://miro.medium.com/max/2120/1*p6PgpOV74U_qLrzr-1_4Zg.png'/>

#### CLS Embeddings

Поскольку преобразователи представляют собой контекстную модель, идея состоит в том, что токен [CLS] захватил бы весь контекст и был бы достаточен для простых последующих задач, таких как классификация.

In [34]:
out['last_hidden_state'][:, 0].shape

torch.Size([6, 768])

In [35]:
reg_head_clstoken_embeddings = nn.Linear(768, 1)(out['last_hidden_state'][:, 0])
reg_head_clstoken_embeddings 

tensor([[-0.3017],
        [-0.3277],
        [-0.2973],
        [-0.3019],
        [-0.3124],
        [-0.3453]], grad_fn=<AddmmBackward>)

#### Mean Pooling

- Step 1: Expand Attention Mask from [batch_size, max_len] to [batch_size, max_len, hidden_size].
- Step 2: Sum Embeddings along max_len axis so now we have [batch_size, hidden_size].
- Step 3: Sum Mask along max_len axis. This is done so that we can ignore padding tokens.
- Step 4: Take Average.

In [67]:
"""
torch.Size([6, 256])
unsqueeze
>> torch.Size([6, 256, 1])
expand we set size . expand(size) and expand for larger size
.epand(6, 256, 768) and expand 768 if set -1 no change size
.epand(-1, -1, 768) get  (6, 256, 768)

nullify unnecessary(by mask where zeros)
out['last_hidden_state'] * att_mask_exp
.sum
>> torch.Size([6, 768])
next we get sum by len
.clamp set values in range
and find mean
>> torch.Size([6, 768])
"""
att_mask_exp = tkzr['attention_mask'].unsqueeze(-1).expand(out['last_hidden_state'].size()).float()
sum_embeddings = torch.sum(out['last_hidden_state'] * att_mask_exp, 1)
sum_mask  = att_mask_exp.sum(1)
sum_mask = torch.clamp(sum_mask, min=1e-9)
mean_embeddings = sum_embeddings / sum_mask
mean_embeddings.shape

torch.Size([6, 768])

In [66]:
nn.Linear(768, 1)(mean_embeddings)

tensor([[-0.0307],
        [-0.0165],
        [ 0.0086],
        [ 0.0197],
        [ 0.0041],
        [-0.0188]], grad_fn=<AddmmBackward>)

#### Max Pooling

In [71]:
att_mask_exp = tkzr['attention_mask'].unsqueeze(-1).expand(out['last_hidden_state'].size()).float()
# Set padding tokens to large negative value
out['last_hidden_state'][att_mask_exp == 0] = -1e9
max_embeddings = torch.max(out['last_hidden_state'], 1)[0]
nn.Linear(768, 1)(max_embeddings)

tensor([[-0.4185],
        [-0.2180],
        [-0.2681],
        [-0.3354],
        [-0.2616],
        [-0.4914]], grad_fn=<AddmmBackward>)

#### Mean-Max Pooling (Head)
<img src= 'https://media.springernature.com/original/springer-static/image/chp%3A10.1007%2F978-3-030-58323-1_23/MediaObjects/498432_1_En_23_Fig1_HTML.png'/>

Сначала мы находим вложения среднего и максимального пула, а затем объединяем их, чтобы получить окончательное представление, которое вдвое превышает скрытый размер.

In [80]:
# var1
mean_embeddings = torch.mean(out['last_hidden_state'], 1)
_, max_embeddings = torch.max(out['last_hidden_state'], 1)
mean_max_embeddings = torch.cat((mean_embeddings, max_embeddings), 1)
nn.Linear(768 * 2, 1)(mean_max_embeddings)

tensor([[  1.9549],
        [ 58.3857],
        [-20.5326],
        [-12.3191],
        [ -5.8741],
        [ 55.0666]], grad_fn=<AddmmBackward>)

In [74]:
# var2
mean_max_embeddings = torch.cat((max_embeddings, mean_embeddings), 1)
nn.Linear(768 * 2, 1)(mean_max_embeddings)

tensor([[-0.2407],
        [-0.3398],
        [-0.3508],
        [-0.3531],
        [-0.2457],
        [-0.3942]], grad_fn=<AddmmBackward>)